In [ ]:
# HW1-Problem 2. Softmax classification without using library

# Import library
import numpy as np #수치계산
import tensorflow as tf #딥러닝
import pandas as pd #데이터 처리
from sklearn.preprocessing import OneHotEncoder #원-핫 인코딩

# data load
df = pd.read_csv('triangular.csv')


# one-hot encocding : 범주형 데이터를 수치형 데이터로 변환하는 방법. 각 범주를 하나의 독립된 차원으로 만들고, 해당 범주에 속하는 데이터에만 1을, 나머지 범주에는 0을 부여
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False) #객체생성. sparse_output=False는 결과를 밀집행렬 (dense matrix)로 반환하도록 설정함
encoded_data = encoder.fit_transform(df[['class']]) #df에서 class열을 추출하고, 원-핫인코딩을 수행하여 희소 행렬을 반환
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out()) #웟-핫 인코딩된 데이터를 DataFrame으로 반환하고, 컬럼 이름을 설정함
df = pd.concat([df, encoded_df], axis=1) #원본 df와 원-핫인코딩된 encoded_df를 열 방향으로 합쳐서 새로운 dataframe을 생성


# 10개의 data만 선택
x_data = np.array(df.iloc[1:11,0:3], dtype=np.float32) #df에서 행 1~10까지 열0~2까지 선택. 이는 입력데이터(features)에 해당하며 x_data에 할당됨
y_data = np.array(df.iloc[1:11,4:], dtype=np.float32) #df에서 행1~10까지 열4번째부터 마지막까지 선택. 이는 출력데이터(lable)에 해당하며 y_data에 할당됨
#np.array(): 선택된 데이터를 NumPy 배열로 변환

# Model parameter
Wl = tf.Variable(tf.random.normal([3, 3])) #가중치W와 편향b를 랜덤하게 초기화
bl = tf.Variable(tf.random.normal([3]))

Wn = tf.Variable(tf.random.normal([3, 3]))
bn = tf.Variable(tf.random.normal([3]))
#tf.Variable: TensorFlow에서 학습 가능한 변수를 생성하는 함수
#tf.random.normal: 평균이 0이고 표준편차가 1인 정규 분포에서 랜덤한 값을 생성
#[3, 3]: 가중치 행렬의 크기를 의미. 입력 데이터의 차원이 3이고, 출력 클래스가 3개라고 가정하면 3x3 행렬이 됨
#[3]: 편향 벡터의 크기를 의미. 출력 클래스가 3개이므로 3개의 편향 값이 필요



# learning rate
learning_rate = 0.01
# Softmax classifier
def softmax_classifier(W,b): #이 함수는 모델의 가중치(W)와 편향(b)를 입력으로 받아 학습을 수행하고, 업데이트된 가중치와 편향을 반환
  with tf.GradientTape() as tape: #TensorFlow의 자동 미분 기능을 활용하기 위해 tf.GradientTape() 컨텍스트 매니저를 생성. 이 블록 내에서 수행되는 연산에 대한 그래프를 기록하여 이후에 기울기를 계산할 수 있음.

    # cost/loss function using softmax classifier
    reg = tf.matmul(x_data,W) + b #입력 데이터 x_data와 가중치 행렬 W를 행렬 곱한 후, 편향 b를 더하여 선형 모델의 출력값 reg을 계산
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=reg, labels=y_data)) #logits는 선형 모델의 출력값이고, labels는 실제 레이블
#tf.nn.softmax_cross_entropy_with_logits : 이함수는 예측값과 실제 레이블 사이의 차이를 측정함

    # Weight update
    gradients = tape.gradient(cost, [W,b]) #tape.gradient 함수를 사용하여 손실 함수에 대한 가중치와 편향의 기울기를 계산
    tf.optimizers.SGD(learning_rate).apply_gradients(zip(gradients,[W,b])) #계산된 기울기를 이용하여 가중치와 편향을 업데이트

    return W,b


def softmax_classifier_nolib(W,b): #TensorFlow의 고수준 API를 최대한 사용하지 않고, 소프트맥스 분류기를 직접 구현한 것
  with tf.GradientTape() as tape:

    # cost/loss function using softmax classifier without library
    reg = tf.matmul(x_data,W) + b #입력 데이터 x_data와 가중치 행렬 W를 행렬 곱한 후, 편향 b를 더하여 선형 모델의 출력값 reg를 계산 / 각 클래스에 대한 로그확률을 의미함

    # max_reg를 모든 예측값(reg)에서 빼줌으로써
    # 지수 함수 값이 매우 커지는 현상을 방지해 줍니다.
    #소프트맥스 함수 계산 시 수치적 안정성을 높이기 위한 트릭
    max_reg = tf.reduce_max(reg, axis=1, keepdims=True) #reg에서 각 행(샘플)의 최댓값을 찾아 max_reg에 저장. 지수 함수의 값이 너무 커져 오버플로우가 발생하는 것을 방지하기 위해 모든 값에서 최댓값을 빼줌
    prob = tf.exp(reg - max_reg) / tf.reduce_sum(tf.exp(reg - max_reg), axis=1, keepdims=True) #소프트맥스 함수를 직접 구현한 부분
    #reg - max_reg를 지수 함수에 적용하여 각 클래스에 대한 비례적인 값을 계산
    #이 값들을 각 행(샘플)의 합으로 나누어 각 클래스에 대한 확률을 계산. 이렇게 하면 각 클래스에 대한 확률의 합이 1이 되도록 정규화됨
    cost = tf.reduce_mean(-tf.reduce_sum(y_data * tf.math.log(prob), axis=1))
    #y_data는 정답 레이블, prob는 예측 확률임. 정답 클래스에 대한 확률의 로그 값에 마이너스를 곱한 후, 모든 샘플에 대한 평균을 구함. 이 값이 작을수록 모델의 예측이 정확한 것임.

    # Weight update
    # Gradient descent를 구현하세요.
    gradients = tape.gradient(cost, [W,b]) #cost에 대한 W와 b의 기울기를 계산
    tf.optimizers.SGD(learning_rate).apply_gradients(zip(gradients,[W,b])) #경사 하강법(SGD)을 사용하여 계산된 기울기를 이용해 가중치와 편향을 업데이트

    return W,b



# Learning
for step in range(5001):

  Wl,bl = softmax_classifier(Wl,bl) # Softmax library를 사용한 classifier
  Wn,bn = softmax_classifier_nolib(Wn,bn) # library 사용 없이 softmax 확률값 및 cost function을 통한 classifier

  if step%500 == 0: #500번마다 학습 과정의 결과를 출력

   # Softmax using library
   model_LC = tf.matmul(x_data,Wl) + bl
   model_lib = tf.argmax(tf.nn.softmax(model_LC),1)

   # cost accuracy 계산
   cost_lib = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model_LC, labels=y_data))
   # tf.nn.softmax_cross_entropy_with_logits: 이 함수는 소프트맥스 크로스 엔트로피 손실 함수를 계산
   # 크로스 엔트로피는 두 확률 분포 사이의 거리를 측정하는 함수로, 모델의 예측 확률과 실제 레이블 사이의 차이를 나타냄.
   # logits=model_LC: model_LC는 선형 모델의 출력값으로, 각 클래스에 대한 로그 확률을 나타냄.
   # labels=y_data: 실제 레이블 데이터.
   # tf.reduce_mean: 배치 평균을 계산. 즉, 모든 샘플에 대한 손실 값의 평균을 구함
   accu_lib = tf.reduce_mean(tf.cast(tf.equal(model_lib, tf.argmax(y_data,1)),tf.float32))
   # tf.equal(model_lib, tf.argmax(y_data,1)): 예측 결과 model_lib와 실제 레이블 y_data의 최댓값 인덱스가 같은지 비교하여 불리언 값을 반환
   # tf.cast(..., tf.float32): 불리언 값을 부동소수점 값으로 변환. True는 1로, False는 0으로 변환됩니다.
   # tf.reduce_mean: 평균을 계산하여 정확도를 구합니다.


   # Softmax without library
   model_LC = tf.matmul(x_data,Wn) + bn #x_data와 Wn, bn을 이용하여 선형 모델의 출력값 model_LC를 계산
   model_nolib = tf.argmax(tf.nn.softmax(model_LC),1)
   #model_LC에 소프트맥스 함수를 적용하여 각 클래스에 대한 확률을 계산
   #tf.argmax를 사용하여 가장 높은 확률을 가진 클래스의 인덱스를 찾아 예측 결과 model_nolib로 저장합니다.

   # cost accuracy 계산
   cost_nolib = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model_LC, labels=y_data)) #소프트맥스 크로스 엔트로피 손실 함수를 사용하여 예측 결과와 실제 레이블 사이의 차이를 계산
   accu_nolib = tf.reduce_mean(tf.cast(tf.equal(model_nolib, tf.argmax(y_data,1)),tf.float32)) #예측 결과와 실제 레이블이 일치하는 비율을 계산
   # tf.equal을 사용하여 예측 결과와 실제 레이블이 같은지 비교
   # tf.cast를 사용하여 불리언 값을 부동소수점 값으로 변환.
   # tf.reduce_mean을 사용하여 평균을 계산하여 정확도를 구함


   print(step, 'Accuracy(lib) : ', accu_lib.numpy(), 'Cost(lib) : ', cost_lib.numpy(), 'Accuracy(no lib) : ', accu_nolib.numpy(), 'Cost(no lib) : ', cost_nolib.numpy())


0 Accuracy(lib) :  0.6 Cost(lib) :  9.917005 Accuracy(no lib) :  0.0 Cost(no lib) :  7.8325295
500 Accuracy(lib) :  0.9 Cost(lib) :  0.31605738 Accuracy(no lib) :  0.8 Cost(no lib) :  0.5564135
1000 Accuracy(lib) :  1.0 Cost(lib) :  0.2623035 Accuracy(no lib) :  0.8 Cost(no lib) :  0.39605188
1500 Accuracy(lib) :  1.0 Cost(lib) :  0.23057334 Accuracy(no lib) :  0.9 Cost(no lib) :  0.3234303
2000 Accuracy(lib) :  1.0 Cost(lib) :  0.20820025 Accuracy(no lib) :  1.0 Cost(no lib) :  0.27838486
2500 Accuracy(lib) :  1.0 Cost(lib) :  0.1910817 Accuracy(no lib) :  1.0 Cost(no lib) :  0.24696672
3000 Accuracy(lib) :  1.0 Cost(lib) :  0.17736182 Accuracy(no lib) :  1.0 Cost(no lib) :  0.22351018
3500 Accuracy(lib) :  1.0 Cost(lib) :  0.16601744 Accuracy(no lib) :  1.0 Cost(no lib) :  0.2051675
4000 Accuracy(lib) :  1.0 Cost(lib) :  0.15641858 Accuracy(no lib) :  1.0 Cost(no lib) :  0.19032648
4500 Accuracy(lib) :  1.0 Cost(lib) :  0.14814876 Accuracy(no lib) :  1.0 Cost(no lib) :  0.17799947
50